In [ ]:
# !pip install noisereduce

In [1]:
import pandas as pd
import numpy as np
import librosa
import librosa.display
from audio_augmentation import add_whitenoise, shift_audio, stratch_audio
import noisereduce as nr
import glob
from genmfcc import gen_mfcc
import os
import IPython.display as ipd

In [5]:
sr = 16000

In [2]:
def file_transform(sig):
    global sr
    sig_lst = []
    sig=nr.reduce_noise(y=sig, sr=sr, stationary=False)

    sig_wn = add_whitenoise(sig)

    sig_strech1 = stratch_audio(sig_wn, rate=np.random.randint(1, 25) / 10)

    sig_roll = shift_audio(sig_strech1, shift=np.random.randint(-9, 9)*sr)

    return sig_roll

In [3]:
files_else = pd.read_csv("./train_df_최종본_15세미만_65세이상.csv")
files_else.head(3)

,uuid,cough_detected,SNR,gender,man,respiratory_condition,fever_muscle_pain,COVID-19,healthy,symptomatic,15세미만,15세이상65세미만,65세이상
0,009b1e22-f451-40ae-9cbf-9f1ff5dec022,0.9743,16.461916,male,1,0,0,0,1,0,1,0,0
1,02bcadc9-b1d1-41d4-9660-2b70216cb81b,0.7553,15.571817,male,1,0,0,0,1,0,1,0,0
2,040f3aa7-2afc-4ca4-925a-e7e25591fe56,0.9688,13.107982,male,1,0,0,0,1,0,1,0,0


## 1. 15세 이상 65세 이하 데이터 처리방법

In [ ]:
filepath = "../public_dataset/processed_dataset/"
files_1565 = pd.read_csv("../public_dataset/train_df_1565.csv")

mfcc_lst = []
Y_data=[]

for i in files_1565.index:
    sig, sr = librosa.load(filepath + files_1565.loc[i, "uuid"] + ".wav", sr = sr)
    for _ in range(4):
        mfcc_lst.append( gen_mfcc( file_transform(sig) ) )

    for _ in range(4):
        Y_data.append(list(files_1565.loc[i, 'COVID-19':'healthy'].values))

## 2. 기타(15세 미만 65세 이상 데이터) 데이터 처리 방법

In [9]:
import warnings
warnings.filterwarnings(action='ignore')

In [10]:
filepath = "../public_dataset/processed_dataset/"
files_else = pd.read_csv("./train_df_최종본_15세미만_65세이상.csv")

mfcc_lst_other = []
Y_data_other=[]

for i in files_else.index:
    sig, sr = librosa.load(filepath + files_else.loc[i, "uuid"] + ".wav", sr = sr)
    for _ in range(18):
        mfcc_lst_other.append( gen_mfcc( file_transform(sig) ) )

    for _ in range(18):
        Y_data_other.append(list(files_else.loc[i, 'COVID-19':'healthy'].values))

In [37]:
import pickle


# pickle 파일로 저장
with open('mfcc_other.pickle', 'wb') as f:
    pickle.dump(mfcc_lst_other, f, pickle.HIGHEST_PROTOCOL)

In [43]:
with open('y_feature_other.pickle', 'wb') as f:
    pickle.dump(Y_data_other, f, pickle.HIGHEST_PROTOCOL)